In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


k_fold = 10
seeds = [192760, 1393659, 1269293, 138973, 931308, 1088652, 1361151, 1456105, 820535, 1240839]


In [3]:
data_train_path = "Dataset/data_train.csv"
data_test_path = "Dataset/data_target_users_test.csv"

In [4]:
df = pd.read_csv(data_train_path)
df_test = pd.read_csv(data_test_path)

df.columns = ["UserID", "ItemID", "Interaction"]
df_test.columns = ["UserID"]

In [5]:
df

,UserID,ItemID,Interaction
0,1,7,1.0
1,1,15,1.0
2,1,16,1.0
3,1,133,1.0
4,1,161,1.0
...,...,...,...
478725,13024,13605,1.0
478726,13024,13823,1.0
478727,13024,15122,1.0
478728,13024,18185,1.0


In [6]:
metric_to_optimize="MAP"
cutoff_to_optimize=10

In [7]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

Number of users: 12638, Number of Items: 22222, Min rating: 0.0, Max rating: 1.0


In [8]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample


urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

urm_train_validation = []
urm_test = []
urm_train = []
urm_validation = []

for i in range(len(seeds)):
    a, b = split_train_in_two_percentage_global_sample(urm_all, train_percentage = 0.80,seed=seeds[i])
    c, d = split_train_in_two_percentage_global_sample(a, train_percentage = 0.80,seed=seeds[i])
    urm_train_validation.append(a)
    urm_test.append(b)
    urm_train.append(c)
    urm_validation.append(d)



In [10]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = []
for u in urm_validation:
    evaluator_validation.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2581 (20.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2598 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2621 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2601 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2649 (21.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2614 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2608 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2586 (20.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2593 

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = []
for u in urm_test:
    evaluator_test.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2146 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2177 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2157 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2132 (16.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2149 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2129 (16.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2228 (17.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2183 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2140 

In [12]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

recommenderClass = ItemKNNCFRecommender



In [13]:
import optuna

# Replace the following with your MySQL database connection details
host = "db-mysql-nyc3-14396-do-user-15539286-0.c.db.ondigitalocean.com"
port = 25060
database_name = "recsys_optuna"
username = "doadmin"
password = "AVNS_EIQ1D-GbDGJWZPsN3Mt"

# Create an SQLAlchemy engine
mysql_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}"
storage = optuna.storages.RDBStorage(url=mysql_url)

In [14]:
rp3_study = optuna.load_study(study_name="RP3_Hybrid_last", storage=storage)
p3_study = optuna.load_study(study_name="P3_Hybrid_last", storage=storage)
slim_study = optuna.load_study(study_name="SLIM_Hybrid_last", storage=storage)
knn_study = optuna.load_study(study_name="KNN_Hybrid_last", storage=storage)
mf_study = optuna.load_study(study_name="NMF_Hybrid_last", storage=storage)


In [15]:
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.Hybrid.GeneralizedLinearHybridRecommender import GeneralizedLinearHybridRecommender

slims = []
rp3s = []
p3s = []
knns = []
nmfs = []

for i in range(k_fold):
    slim = MultiThreadSLIM_SLIMElasticNetRecommender(urm_train[i])
    slim.fit(workers=8, **slim_study.best_params)
    slims.append(slim)

    rp3 = RP3betaRecommender(urm_train[i])
    rp3.fit(**rp3_study.best_params)
    rp3s.append(rp3)

    p3 = P3alphaRecommender(urm_train[i])
    p3.fit(**p3_study.best_params)
    p3s.append(p3)

    knn = ItemKNNCFRecommender(urm_train[i])
    knn.fit(**knn_study.best_params)
    knns.append(knn)

    nmf = NMFRecommender(urm_train[i])
    nmf.fit(**mf_study.best_params)
    nmfs.append(nmf)

SLIMElasticNetRecommender: URM Detected 508 ( 4.0%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 332 ( 1.5%) items with no interactions.


100%|█████████▉| 22216/22222 [00:29<00:00, 744.76it/s] 


RP3betaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
RP3betaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 9472.88 column/sec. Elapsed time 2.35 sec
P3alphaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
P3alphaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 10479.71 column/sec. Elapsed time 2.12 sec
ItemKNNCFRecommender: URM Detected 508 ( 4.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 332 ( 1.5%) items with no interactions.
Similarity column 22222 (100.0%), 13554.67 column/sec. Elapsed time 1.64 sec
NMFRecommender: URM Detected 508 ( 4.0%) users with no interactions.
NMFRecommender: URM Detected 332 ( 1.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 14.13 sec
SLIMElasticNetRecommender: URM Detecte

100%|█████████▉| 22216/22222 [00:30<00:00, 736.89it/s] 


RP3betaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
RP3betaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 9596.34 column/sec. Elapsed time 2.32 sec
P3alphaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
P3alphaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 10274.26 column/sec. Elapsed time 2.16 sec
ItemKNNCFRecommender: URM Detected 440 ( 3.5%) users with no interactions.
ItemKNNCFRecommender: URM Detected 325 ( 1.5%) items with no interactions.
Similarity column 22222 (100.0%), 13503.48 column/sec. Elapsed time 1.65 sec
NMFRecommender: URM Detected 440 ( 3.5%) users with no interactions.
NMFRecommender: URM Detected 325 ( 1.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 10.50 sec
SLIMElasticNetRecommender: URM Detecte

...

RP3betaRecommender: URM Detected 471 ( 3.7%) users with no interactions.
RP3betaRecommender: URM Detected 362 ( 1.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 9412.71 column/sec. Elapsed time 2.36 sec
P3alphaRecommender: URM Detected 471 ( 3.7%) users with no interactions.
P3alphaRecommender: URM Detected 362 ( 1.6%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 10484.33 column/sec. Elapsed time 2.12 sec
ItemKNNCFRecommender: URM Detected 471 ( 3.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 362 ( 1.6%) items with no interactions.
Similarity column 22222 (100.0%), 13941.16 column/sec. Elapsed time 1.59 sec
NMFRecommender: URM Detected 471 ( 3.7%) users with no interactions.
NMFRecommender: URM Detected 362 ( 1.6%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 9.43 sec
SLIMElasticNetRecommender: URM Detected

100%|█████████▉| 22216/22222 [00:30<00:00, 725.10it/s]


RP3betaRecommender: URM Detected 478 ( 3.8%) users with no interactions.
RP3betaRecommender: URM Detected 329 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 9456.40 column/sec. Elapsed time 2.35 sec
P3alphaRecommender: URM Detected 478 ( 3.8%) users with no interactions.
P3alphaRecommender: URM Detected 329 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 10419.18 column/sec. Elapsed time 2.13 sec
ItemKNNCFRecommender: URM Detected 478 ( 3.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 329 ( 1.5%) items with no interactions.
Similarity column 22222 (100.0%), 13797.02 column/sec. Elapsed time 1.61 sec
NMFRecommender: URM Detected 478 ( 3.8%) users with no interactions.
NMFRecommender: URM Detected 329 ( 1.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 13.71 sec
SLIMElasticNetRecommender: URM Detecte

100%|█████████▉| 22216/22222 [00:30<00:00, 724.01it/s]


RP3betaRecommender: URM Detected 460 ( 3.6%) users with no interactions.
RP3betaRecommender: URM Detected 348 ( 1.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 9561.45 column/sec. Elapsed time 2.32 sec
P3alphaRecommender: URM Detected 460 ( 3.6%) users with no interactions.
P3alphaRecommender: URM Detected 348 ( 1.6%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 10442.94 column/sec. Elapsed time 2.13 sec
ItemKNNCFRecommender: URM Detected 460 ( 3.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 348 ( 1.6%) items with no interactions.
Similarity column 22222 (100.0%), 13834.26 column/sec. Elapsed time 1.61 sec
NMFRecommender: URM Detected 460 ( 3.6%) users with no interactions.
NMFRecommender: URM Detected 348 ( 1.6%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 11.09 sec


In [16]:
hybrids = []

for i in range(k_fold):
    hybrid = GeneralizedLinearHybridRecommender(urm_train[i], recommenders=[slims[i], rp3s[i], p3s[i], knns[i], nmfs[i]])
    hybrids.append(hybrid)

SLIMElasticNetRP3betaP3alphaItemKNNCFNMFHybridRecommender: URM Detected 508 ( 4.0%) users with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFNMFHybridRecommender: URM Detected 332 ( 1.5%) items with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFNMFHybridRecommender: URM Detected 440 ( 3.5%) users with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFNMFHybridRecommender: URM Detected 325 ( 1.5%) items with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFNMFHybridRecommender: URM Detected 479 ( 3.8%) users with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFNMFHybridRecommender: URM Detected 345 ( 1.6%) items with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFNMFHybridRecommender: URM Detected 451 ( 3.6%) users with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFNMFHybridRecommender: URM Detected 311 ( 1.4%) items with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFNMFHybridRecommender: URM Detected 449 ( 3.6%) users with no intera

In [17]:
study = optuna.create_study(study_name="LINEAR_TOPPOP_Hybrid_last", direction="maximize", storage=storage, load_if_exists=True, sampler=optuna.samplers.TPESampler())

[I 2024-01-10 10:01:48,884] A new study created in RDB with name: LINEAR_Hybrid_last


In [19]:
def objective(trial):

    alphas = []
    for i in range(len(hybrids[0].recommenders)):
        alphas.append(trial.suggest_float(f"alpha_{i}", 0, 1))

    MAP = 0
    for i in range(k_fold):
        recommender = hybrids[i]
        recommender.fit(alphas=alphas)

        result_dict, _ = evaluator_validation[i].evaluateRecommender(recommender)
        print(f"MAP: {result_dict['MAP'].item()}, RECALL: {result_dict['RECALL'].item()}")
        MAP += result_dict["MAP"].item()
    
    MAP /= k_fold
    return MAP

In [23]:
study.optimize(objective, n_trials=200)

EvaluatorHoldout: Processed 10057 (100.0%) in 7.87 sec. Users per second: 1277
MAP: 0.030150317791761787, RECALL: 0.1183030190936985
EvaluatorHoldout: Processed 10040 (100.0%) in 8.12 sec. Users per second: 1237
MAP: 0.030466605799026415, RECALL: 0.11630107734564053
EvaluatorHoldout: Processed 10017 (100.0%) in 8.06 sec. Users per second: 1243
MAP: 0.029730042261489082, RECALL: 0.11456548494643881
EvaluatorHoldout: Processed 10037 (100.0%) in 7.91 sec. Users per second: 1268
MAP: 0.029969549175985623, RECALL: 0.11792955236529927
EvaluatorHoldout: Processed 9989 (100.0%) in 7.61 sec. Users per second: 1312
MAP: 0.03053717025235714, RECALL: 0.12192011943522219
EvaluatorHoldout: Processed 10024 (100.0%) in 7.75 sec. Users per second: 1293
MAP: 0.030301118585236944, RECALL: 0.11792474464013306
EvaluatorHoldout: Processed 10030 (100.0%) in 8.16 sec. Users per second: 1229
MAP: 0.030544604282391186, RECALL: 0.12186609672832321
EvaluatorHoldout: Processed 10052 (100.0%) in 7.96 sec. Users per

[I 2024-01-10 12:34:46,451] Trial 102 finished with value: 0.030227725879068516 and parameters: {'alpha_0': 0.5845979005895502, 'alpha_1': 0.9186732560164836, 'alpha_2': 0.3067220394343694, 'alpha_3': 0.15717607493000826, 'alpha_4': 0.4886211827284999}. Best is trial 53 with value: 0.031146826060747413.


EvaluatorHoldout: Processed 10057 (100.0%) in 7.67 sec. Users per second: 1311
MAP: 0.030843260873339568, RECALL: 0.11915803786516443
EvaluatorHoldout: Processed 10040 (100.0%) in 8.08 sec. Users per second: 1243
MAP: 0.03127159615506259, RECALL: 0.11835567102036558
EvaluatorHoldout: Processed 10017 (100.0%) in 8.17 sec. Users per second: 1226
MAP: 0.030589814775199863, RECALL: 0.1157523998231087
EvaluatorHoldout: Processed 10037 (100.0%) in 7.69 sec. Users per second: 1305
MAP: 0.030930398794302552, RECALL: 0.11873052056054376
EvaluatorHoldout: Processed 9989 (100.0%) in 7.68 sec. Users per second: 1301
MAP: 0.03111901663258179, RECALL: 0.12355831838939316
EvaluatorHoldout: Processed 10024 (100.0%) in 7.54 sec. Users per second: 1330
MAP: 0.031344463763159115, RECALL: 0.11874348872019339
EvaluatorHoldout: Processed 10030 (100.0%) in 7.33 sec. Users per second: 1368
MAP: 0.03139093433983794, RECALL: 0.12369849519225322
EvaluatorHoldout: Processed 10052 (100.0%) in 7.85 sec. Users per s

[I 2024-01-10 12:36:16,651] Trial 103 finished with value: 0.03104250174281698 and parameters: {'alpha_0': 0.6072058501151534, 'alpha_1': 0.9770724730349598, 'alpha_2': 0.2849113047907532, 'alpha_3': 0.07958635639774483, 'alpha_4': 0.11728837717392554}. Best is trial 53 with value: 0.031146826060747413.


EvaluatorHoldout: Processed 10057 (100.0%) in 7.49 sec. Users per second: 1343
MAP: 0.03090448727964909, RECALL: 0.11925443708845503
EvaluatorHoldout: Processed 10040 (100.0%) in 7.43 sec. Users per second: 1351
MAP: 0.03131256719155153, RECALL: 0.11834399911664505
EvaluatorHoldout: Processed 10017 (100.0%) in 7.68 sec. Users per second: 1304
MAP: 0.030587675554731856, RECALL: 0.11578450918472592
EvaluatorHoldout: Processed 10037 (100.0%) in 7.17 sec. Users per second: 1399
MAP: 0.03097972422671059, RECALL: 0.1190727800346781
EvaluatorHoldout: Processed 9989 (100.0%) in 7.19 sec. Users per second: 1389
MAP: 0.03115093666525255, RECALL: 0.12284912570182858
EvaluatorHoldout: Processed 10024 (100.0%) in 7.58 sec. Users per second: 1322
MAP: 0.031393073290769, RECALL: 0.11828849078269443
EvaluatorHoldout: Processed 10030 (100.0%) in 7.72 sec. Users per second: 1300
MAP: 0.03146201870578772, RECALL: 0.12389301274760084
EvaluatorHoldout: Processed 10052 (100.0%) in 7.92 sec. Users per second

...

EvaluatorHoldout: Processed 10057 (100.0%) in 7.46 sec. Users per second: 1349
MAP: 0.030928718210959685, RECALL: 0.11962136561083608
EvaluatorHoldout: Processed 10040 (100.0%) in 7.63 sec. Users per second: 1316
MAP: 0.03141870612786979, RECALL: 0.11843171774969342
EvaluatorHoldout: Processed 10017 (100.0%) in 7.76 sec. Users per second: 1291
MAP: 0.03063068973221746, RECALL: 0.11622802846010276
EvaluatorHoldout: Processed 10037 (100.0%) in 7.46 sec. Users per second: 1346
MAP: 0.031026744695420824, RECALL: 0.1191398396153836
EvaluatorHoldout: Processed 9989 (100.0%) in 7.79 sec. Users per second: 1283
MAP: 0.031224807605826868, RECALL: 0.12331258622485525
EvaluatorHoldout: Processed 10024 (100.0%) in 7.07 sec. Users per second: 1418
MAP: 0.03137155746842512, RECALL: 0.1186867104189563
EvaluatorHoldout: Processed 10030 (100.0%) in 7.63 sec. Users per second: 1314
MAP: 0.03151793033597646, RECALL: 0.12394044474337869
EvaluatorHoldout: Processed 10052 (100.0%) in 7.60 sec. Users per sec

[I 2024-01-10 12:40:42,643] Trial 106 finished with value: 0.031102904994968927 and parameters: {'alpha_0': 0.4556775828443981, 'alpha_1': 0.9793970456793708, 'alpha_2': 0.223187193795333, 'alpha_3': 0.0912898015817012, 'alpha_4': 0.0490794871804618}. Best is trial 53 with value: 0.031146826060747413.


EvaluatorHoldout: Processed 10057 (100.0%) in 7.31 sec. Users per second: 1376
MAP: 0.03074645157522778, RECALL: 0.11869089472249807
EvaluatorHoldout: Processed 10040 (100.0%) in 6.91 sec. Users per second: 1452
MAP: 0.031215985265288237, RECALL: 0.11799191001335634
EvaluatorHoldout: Processed 10017 (100.0%) in 7.85 sec. Users per second: 1275
MAP: 0.03070390257197716, RECALL: 0.11635963258108838
EvaluatorHoldout: Processed 10037 (100.0%) in 7.45 sec. Users per second: 1347
MAP: 0.031010681114796084, RECALL: 0.12013586082715348
EvaluatorHoldout: Processed 9989 (100.0%) in 7.77 sec. Users per second: 1285
MAP: 0.030953060271060234, RECALL: 0.12337779205533406
EvaluatorHoldout: Processed 10024 (100.0%) in 7.86 sec. Users per second: 1275
MAP: 0.031189870501273257, RECALL: 0.11906149755537745
EvaluatorHoldout: Processed 10030 (100.0%) in 7.36 sec. Users per second: 1363
MAP: 0.0313625314532594, RECALL: 0.12355143481002356
EvaluatorHoldout: Processed 10052 (100.0%) in 7.54 sec. Users per s

[W 2024-01-10 12:41:55,479] Trial 107 failed with parameters: {'alpha_0': 0.3819594899768928, 'alpha_1': 0.9789231418363344, 'alpha_2': 0.222977036980774, 'alpha_3': 0.19725729918802395, 'alpha_4': 0.04756946758555092} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_387424/2033201123.py", line 12, in objective
    result_dict, _ = evaluator_validation[i].evaluateRecommender(recommender)
  File "/home/japo/RecSys-Challenge/Evaluation/Evaluator.py", line 276, in evaluateRecommender
    results_dict = self._run_evaluation_on_selected_users(recommender_object, self.users_to_evaluate)
  File "/home/japo/RecSys-Challenge/Evaluation/Evaluator.py", line 476, in _run_evaluation_on_selected_users
    recommended_items_batch_list, scores_batch = recommender_object.recommen

KeyboardInterrupt: 

In [ ]:
best_MAP_test = 0

for i in range(k_fold):
    recommender = recommenderClass(urm_train[i])
    recommender.fit(**study.best_params)

    result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

    best_MAP_test += result_dict["MAP"].item()

best_MAP_test /= k_fold
best_MAP_test

In [ ]:
best_MAP_test = 0

for i in range(k_fold):
    recommender = recommenderClass(urm_train_validation[i])
    recommender.fit(**study.best_params)

    result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

    best_MAP_test += result_dict["MAP"].item()

best_MAP_test /= k_fold
best_MAP_test